In [222]:
import os
import pickle
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
rootdir = 'D:/licencjat/bachelors_thesis_movie_clustering_codes/raw_data/boxoffice_data'
movies_data_dict = {}

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        pickle_path = os.path.join(subdir, file)
        with open(pickle_path, 'rb') as f:
            movie_data = pickle.load(f)
        movie_title = subdir.replace('D:/licencjat/bachelors_thesis_movie_clustering_codes/raw_data/boxoffice_data\\', '')
        movies_data_dict[movie_title] = movie_data

In [345]:
# klucze tego słownika to numery tabel
features_dict = {0:['Domestic Box Office', 'International Box Office', 'Worldwide Box Office'],
                1:['Opening\xa0Weekend:', 'Legs:', 'Domestic Share:', 'Theater counts:', 'Production\xa0Budget:', 'Infl. Adj. Dom. BO'],
                3:['MPAA\xa0Rating:', 'Running Time:', 'Keywords:', 'Source:', 'Genre:', 'Production\xa0Method:', 'Creative\xa0Type:', 'Production Companies:', 'Production Countries:', 'Languages:', 'Franchise:'],
                4:['Records where the movie is in top 5 count']
                }

features_list = features_dict[0] + features_dict[1] + features_dict[3] + features_dict[4]

features_df = pd.DataFrame(index=features_list, columns=movies_data_dict.keys())

In [346]:
# sprawdzam czy zawsze indeksy są niezmienne i odwołują się do tych samych tabel.

for key in movies_data_dict:
    movie_tables = movies_data_dict[key]

    intersection = set(movie_tables[0][0]).intersection(features_dict[0])
    if len(intersection) < 3:
        print('table_0', key, str(len(intersection)))

    intersection = set(movie_tables[1][0]).intersection(features_dict[1])
    if len(intersection) < 5:
        print(intersection)
        print('table_1', key, str(len(intersection)))

    intersection = set(movie_tables[3][0]).intersection(features_dict[3])
    if len(intersection) < 6:
        print('table_3', key, str(len(intersection)))

    intersection = set(movie_tables[4].columns).intersection(set(['Record', 'Rank', 'Amount', 'ChartDate', 'Days InRelease']))
    if  len(intersection) < 5:
        print('table_4', key, str(len(intersection)))

# wniosek: wszystko się zgadza!
        

In [349]:
def get_features_from_table(table_num, features_dict, movie_tables, movie_title, features_df):
    table = movie_tables[table_num]
    features_in_table = features_dict[table_num]

    if table_num != 4:
        for feature in features_in_table:
            if feature in list(table[0]):
                features_df.loc[feature, movie_title] = list(table[table[0]==feature][1])[0]
            else:
                features_df.loc[feature, movie_title] = np.nan

    else:
        features_df.loc['Records where the movie is in top 5 count', movie_title] = (table['Rank'] <= 5).sum()

In [350]:
for key in movies_data_dict:
    movie_tables = movies_data_dict[key]
    get_features_from_table(0, features_dict, movie_tables, key, features_df)
    get_features_from_table(1, features_dict, movie_tables, key, features_df)
    get_features_from_table(3, features_dict, movie_tables, key, features_df)
    get_features_from_table(4, features_dict, movie_tables, key, features_df)

In [394]:
features_df_transposed = features_df.T

In [395]:
features_df_transposed.columns

Index(['Domestic Box Office', 'International Box Office',
       'Worldwide Box Office', 'Opening Weekend:', 'Legs:', 'Domestic Share:',
       'Theater counts:', 'Production Budget:', 'Infl. Adj. Dom. BO',
       'MPAA Rating:', 'Running Time:', 'Keywords:', 'Source:', 'Genre:',
       'Production Method:', 'Creative Type:', 'Production Companies:',
       'Production Countries:', 'Languages:', 'Franchise:',
       'Records where the movie is in top 5 count'],
      dtype='object')

In [396]:
features_df_transposed[['Domestic Box Office', 'International Box Office', 'Worldwide Box Office', 'Infl. Adj. Dom. BO']] = features_df_transposed[['Domestic Box Office', 'International Box Office', 'Worldwide Box Office', 'Infl. Adj. Dom. BO']].replace('\$','', regex=True).replace(',','', regex=True)

search = []    
for values in features_df_transposed['Opening\xa0Weekend:']:
    search.append(re.search(r'\d+\.*\d*\%', values).group())
features_df_transposed['opening weekend / total gross percentage'] = search
features_df_transposed['opening weekend / total gross percentage'] = features_df_transposed['opening weekend / total gross percentage'].replace('\%', '', regex=True)

features_df_transposed['Opening\xa0Weekend:'] = features_df_transposed['Opening\xa0Weekend:'].replace('\(.*\)', '', regex=True).replace('\$','', regex=True).replace(',','', regex=True)

features_df_transposed['Legs (domestic box office/biggest weekend percentage)'] = features_df_transposed['Legs:'].replace('\s*\(.*\)', '', regex=True)
features_df_transposed = features_df_transposed.drop('Legs:', axis=1)

features_df_transposed['Domestic Share (domestic box office/worldwide percentage)'] = features_df_transposed['Domestic Share:'].replace('\%\s\(.*\)', '', regex=True)
features_df_transposed = features_df_transposed.drop('Domestic Share:', axis=1)

features_df_transposed['Opening theaters'] = features_df_transposed['Theater counts:'].replace('\s.+', '', regex=True).replace(',', '', regex=True)
features_df_transposed['Max theaters'] = features_df_transposed['Theater counts:'].replace('\d+,*\d*\s*opening theaters/', '', regex=True).replace('\s.+', '', regex=True).replace(',', '', regex=True)
features_df_transposed['Weeks average run per theater'] = features_df_transposed['Theater counts:'].replace('\d+,*\d* opening theaters/\d+,*\d* max\. theaters, ', '', regex=True).replace('\s.+', '', regex=True)
features_df_transposed = features_df_transposed.drop('Theater counts:', axis=1)

features_df_transposed['worldwide box office / production budget ratio'] = features_df_transposed['Production\xa0Budget:'].replace('.+ \(worldwide box office is ', '', regex=True).replace(' times production budget\)', '', regex=True)
features_df_transposed['Production\xa0Budget:'] = features_df_transposed['Production\xa0Budget:'].replace('\s.+', '', regex=True).replace('\$', '', regex=True).replace(',', '', regex=True)

features_df_transposed['MPAA\xa0Rating:'] = features_df_transposed['MPAA\xa0Rating:'].replace('\s.+', '', regex=True)

features_df_transposed['Running Time:'] = features_df_transposed['Running Time:'].replace('\s.+', '', regex=True)

features_df_transposed['Franchise:'] = features_df_transposed['Franchise:'].replace('.*Marvel Cinematic Universe.*', 'Marvel Cinematic Universe', regex=True).replace('.*DC Extended Universe.*', 'DC Extended Universe', regex=True).replace('.*X-Men.*', '', regex=True).replace('.*Dark Knight Trilogy.*', 'Dark Knight Trilogy', regex=True).replace('.*Harry Potter.*', 'Harry Potter', regex=True).replace('.*Despicable Me.*', 'Despicable Me', regex=True).replace('.*Peter Jackson\'s Lord of the Rings.*', 'Peter Jackson\'s Lord of the Rings', regex=True).replace('.*Spider-Man.*', 'Spider-Man', regex=True)

In [408]:
features_df_transposed['Keywords:']

2012                         Disaster, End of the World, Extreme Weather, V...
22_Jump_Street               College, Undercover, Football, Narcotics, Budd...
300                          Visual Effects, Historical Battles, Voiceover/...
Aladdin                      Remake, Live Action Cartoon, Wish Granters, Ro...
Alice_in_Wonderland          3-D, Arranged Marriage, Talking Animals, Drago...
                                                   ...                        
X-Men_Days_of_Future_Past    Marvel Comics, Mutants, Time Travel, Robot, Po...
X-Men_Origins_Wolverine                                          Marvel Comics
X-Men_The_Last_Stand         Marvel Comics, Visual Effects, Mutants, 2000s,...
xXx                          Secret Agent, Extreme Sports, Car Chase, Hosta...
Zootopia                     Beat Cops, Confidence Men, Talking Animals, An...
Name: Keywords:, Length: 261, dtype: object

In [411]:
keywords_set = set()
for keywords in features_df_transposed['Keywords:'].dropna():
    keywords_list = keywords.split(', ')
    keywords_set.update(keywords_list)

keywords_set

{'1700s',
 '1910s',
 '1920s',
 '1930s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '2000s',
 '2010 Oscars Best Picture Nominee',
 '2011 Oscars Best Picture Nominee',
 '2013 Oscars Best Picture Nominee',
 '2014 Oscars Best Picture Nominee',
 '2015 Oscars Best Picture Nominee',
 '2016 Oscars Best Picture Nominee',
 '2019 Oscars Best Picture Nominee',
 '2020 Oscars Best Picture Nominee',
 '2020s',
 '3-D',
 '3-D - Post-production Conversion',
 '3-D - Shot in 3-D',
 '3D Re-release',
 'A Cappella',
 'AIDS',
 'Abandoned Child',
 'Academics',
 'Accidental Engagement or Marriage',
 'Accidental Injury',
 'Actors Making Their Directorial Debut',
 'Addiction',
 'Adopted Family',
 'Adult Child Dealing with the Death of a Parent',
 'Afghanistan War',
 'Africa',
 'African-American',
 'Agitprop',
 'Alien Abduction',
 'Alien Invasion',
 'Alternate History',
 'Alternate Reality Game',
 'Alternative Dimensions / Parallel universe',
 'Ancient Egypt',
 'Animal Lead',
 'Animals Gone Bad',
 'Animated Inanimate

In [397]:
features_df_transposed

,Domestic Box Office,International Box Office,Worldwide Box Office,Opening Weekend:,Production Budget:,Infl. Adj. Dom. BO,MPAA Rating:,Running Time:,Keywords:,Source:,...,Languages:,Franchise:,Records where the movie is in top 5 count,opening weekend / total gross percentage,Legs (domestic box office/biggest weekend percentage),Domestic Share (domestic box office/worldwide percentage),Opening theaters,Max theaters,Weeks average run per theater,worldwide box office / production budget ratio
2012,166112167,591565581,757677748,65237614,200000000,201610604,PG-13,158,"Disaster, End of the World, Extreme Weather, V...",Original Screenplay,...,English,NaN,2,39.3,2.55,21.9,3404,3444,6.4,3.8
22_Jump_Street,191719337,139614539,331333876,57071445,50000000,213777619,R,111,"College, Undercover, Football, Narcotics, Budd...",Based on TV,...,English,21 Jump Street,1,29.8,3.36,57.9,3306,3426,6.7,6.6
300,210614939,243546996,454161935,70885301,60000000,278881114,R,115,"Visual Effects, Historical Battles, Voiceover/...",Based on Comic/Graphic Novel,...,English,300,1,33.7,2.97,46.4,3103,3280,6.8,7.6
Aladdin,355559216,695400000,1050959216,91500929,182000000,355559219,PG,128,"Remake, Live Action Cartoon, Wish Granters, Ro...",Remake,...,English,NaN,5,25.7,3.89,33.8,4476,4476,8.0,5.8
Alice_in_Wonderland,334191110,691300000,1025491110,116101023,200000000,385865775,PG,108,"3-D, Arranged Marriage, Talking Animals, Drago...",Based on Fiction Book/Short Story,...,English,Alice in Wonderland,1,34.7,2.88,32.6,3728,3739,7.2,5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X-Men_Days_of_Future_Past,233921534,513941241,747862775,90823660,200000000,260835388,PG-13,131,"Marvel Comics, Mutants, Time Travel, Robot, Po...",Based on Comic/Graphic Novel,...,English,,3,38.8,2.58,31.3,3996,4001,6.0,3.7
X-Men_Origins_Wolverine,179883157,194942603,374825760,85058003,150000000,218498075,PG-13,107,Marvel Comics,Spin-Off,...,English,,1,47.3,2.11,48.0,4099,4102,5.4,2.5
X-Men_The_Last_Stand,234362462,224898484,459260946,102750665,210000000,325960610,PG-13,104,"Marvel Comics, Visual Effects, Mutants, 2000s,...",Based on Comic/Graphic Novel,...,English,,10,43.8,2.28,51.0,3690,3714,5.6,2.2
xXx,141930000,125270000,267200000,44506103,70000000,222544282,PG-13,124,"Secret Agent, Extreme Sports, Car Chase, Hosta...",Original Screenplay,...,English,xXx,0,31.4,3.19,53.1,3374,3536,6.9,3.8
